(Carregar em >|Run para sucessivamente ir executando as células do notebook)

## Instalar e Disponibilizar Aplicações Essenciais

Utilização remota, e.g. Binder: 

  * Mantêr todos os # 

Se fôr a primeira utilização local deste notebook :
  * Caso Julia Pro tenha sido instalada, apenas retirar os # na primeira e na segunda linha
  * Caso contrário, retirar # em todas as linhas (ou apenas naquelas relativas a aplicações ainda não instaladas)

In [ ]:
# import Pkg 
# Pkg.add("TimeSeries")
# Pkg.add("StatsPlots")         
# Pkg.add("Interact")     

In [ ]:
using TimeSeries, StatsPlots, Interact

## 1. Definir Modelo de Nelson-Siegel-Svensson (NSS)

### 1.1 Taxas Spot

$$spot^{\textit{NSS}}(m)= \beta_0+\beta_1\frac{1-\exp(-m/\tau_1)}{m /\tau_1} + \beta_2 \biggl(\frac{1-\exp(-m/\tau_1)}{m/\tau_1}-\exp(-m/\tau_1)\biggr) + \beta_3\biggl(\frac{1-\exp(-m/\tau_2)}{m/\tau_2}-\exp(-m/\tau_2)\biggr)$$

Função: ```spot(tenores, coeficientes_nss)``` 

Exemplo: 
  * ```tenores =``` $[m_1, m_2,\dots, m_6]$ ```= [1,3,5,7,10,15]```
  * ```coeficientes_nss ``` serão lidos de um ficheiro

In [ ]:
function spot(m,nss)           # (i) m representa os tenores (ii) nss, os coeficientes_nss [BETA0, BETA1,...,TAU2]
    ntenores = size(m,1)       # ntenores = número de tenores a calcular 
    if size(nss,2) == 1        # se apenas uma observação, a matriz *nss* passa a vector com 7 linhas e 1 coluna!
        ni = 1                 # ni = número de observações é 1 e não 7
        nss = nss'             # ok, então também temos de transpôr 
    else
        ni = size(nss,1)    
    end  
    s = zeros(ni,ntenores)
    for j = 1:ntenores
        for i = 1:ni   
            s[i,j] = nss[i,1] + nss[i,2]*(1 - exp(-m[j]/nss[i,5]))/(m[j]/nss[i,5]) + 
            nss[i,3]*((1 - exp(-m[j]/nss[i,5]))/(m[j]/nss[i,5]) - exp(-m[j]/nss[i,5])) + 
            nss[i,4]*((1 - exp(-m[j]/nss[i,6]))/(m[j]/nss[i,6]) - exp(-m[j]/nss[i,6])) 
        end
    end
    return s = s /100           # Não apresentar taxas spot em percentagem
end 

### 1.2 Taxas Forward

$$forward^{\textit{NSS}}(m)=\beta_0+\beta_1 \exp(-m/\tau_1) +\beta_2 \frac{m \exp(-m/\tau_1)}{\tau_1}  +\beta_3 \frac{m \exp(-m/\tau_2)}{\tau_2}$$

In [ ]:
function forward(m,nss)         
    ntenores = size(m,1)       
    if size(nss,2) == 1      
        ni = 1                
        nss = nss'      
    else
        ni = size(nss,1)     
    end  
    f = zeros(ni,ntenores)
    for j = 1:ntenores
        for i = 1:ni   
            f[i,j] =  nss[i,1] + nss[i,2]*exp(-m[j]/nss[i,5]) + 
            nss[i,3]*(m[j]/nss[i,5])*exp(-m[j]/nss[i,5]) + 
            nss[i,4]*(m[j]/nss[i,6])*exp(-m[j]/nss[i,6])
        end
    end
    return f = f /100           # Não apresentar taxas forward em percentagem
end 

## 2. Obter Coeficientes do Modelo NSS para Emissões AAA de Dívida Pública em EUR

Iremos ler os coeficientes do modelo de Nelson-Siegel-Svensson para dívida pública soberana AAA em Euros, calculados pelo Banco Central Europeu (BCE). Estão disponiveis no ficheiro ```ETTJ.csv```, desde o início em 2004-09-06 até 2019-03-28. Podemos também obter estes dados a partir da webpage do BCE.   

In [ ]:
TabelaNSS = readtimearray("ETTJ.csv")

Por conveniência, iremos guardar as datas das observações no vetor ```datas(3726x1)``` e criar uma matriz ```MatrizNSS(3726x6)``` apenas com os valores da tabela

In [ ]:
datas = timestamp(TabelaNSS)
MatrizNSS = values(TabelaNSS)

## 3. Evolução da Estrutura Temporal Taxas Juro

### 3.1 Calcular Taxas Spot e Forward pelo Modelo NSS  

In [ ]:
tenores = [0.1,0.2,0.3,0.4,0.5,0.6,0.8,1.0,1.5,2.0,2.5,3.0,4.0,4.5,5,6,7,8,9,10,11,12,14,16,18,20,22,24,26,28,30];

In [ ]:
spot_nss = spot(tenores,MatrizNSS)   

In [ ]:
forward_nss = forward(tenores,MatrizNSS)  

### 3.2 Visualizar a Estrutura Temporal Taxas de Juro Spot e Forward na Data Mais Recente (dos dados)

In [ ]:
nobs = size(spot_nss,1)                        # Número de observações (será utilizado no gráfico interativo)
ntenores = size(tenores,1) 
plot(tenores, spot_nss[nobs,1:ntenores],
    title = "Estrutura Temporal de Taxas de Juro em $(datas[nobs,1])",
    xlabel = "Tenores (em anos)", 
    xticks = 0:5:30,
    ylabel = "Dívida Pública AAA na Eurozona", 
    ylims = (-0.01,0.015),
    yticks = -0.01:0.005:0.015,
    label = ["spot"])
plot!(tenores, forward_nss[nobs,1:ntenores],  legend = :bottomright,  label = ["spot", "forward"])

### 3.3 Evolução da Estrutura Temporal de Taxas de Juro de Acordo com o Modelo NSS

In [ ]:
@manipulate for obs = 1:nobs, curva = [:spot, :forward]
    if curva == :spot
        ettj = spot_nss
        labelv = ["spot"]
    else
        ettj = forward_nss
        labelv = ["forward"]
    end
    plot(tenores, ettj[obs,1:ntenores], 
     label = labelv, 
     legend = :bottomright,
     xlabel = "Tenores (em anos)", 
     xticks = 0:5:30,
     ylabel = "Taxas dívida pública AAA na Eurozona", 
     ylims = (-0.01,0.06),
     yticks = -0.01:0.01:0.06,
     title = "Estrutura Temporal de Taxas de Juro em $(datas[obs,1])")
end
# No gráfico seguinte: teclar spot/forward para selecionar a curva, mover o cursor no topo para selecionar a data 

### 3.4 Instabilidade na Estimativa dos Parâmetros do Modelo NSS 

In [ ]:
p1 = plot(MatrizNSS[:,1],ylabel = "\\beta_0")
p2 = plot(MatrizNSS[:,2],ylabel = "\\beta_1")
p3 = plot(MatrizNSS[:,3],ylabel = "\\beta_2")
p4 = plot(MatrizNSS[:,4],ylabel = "\\beta_3")
p5 = plot(MatrizNSS[:,5],ylabel = "\\tau_1")
p6 = plot(MatrizNSS[:,6],ylabel = "\\tau_2")

plot(p1,p2,p3,p4,p5,p6,layout=(3,2), legend=false)